### Scenario:

A supplier is shipping N different parts to Tesla.

Every part has a known (deterministic) demand every day.  That is, we know how many parts we will need to build the cars scheduled for production that day.

Each part comes in a box with a certain given quantity, and a certain known volume for the box.

Parts are delivered in trucks. The truck cost is essentially fixed per truck, that is regardless of whether a truck is half full or completely full the cost is the same. For simplicity, we can model the truck has having a known max volume capacity, and the sum of the volumes of the parts in the truck has to be less than this capacity. (Obviously this is a simplification, in reality fitting the most number of boxes in a truck is a more complex problem)

Unused parts from one day carry forward to the next day. We must never run out of a part completely, but other than that generally less inventory is preferred over more.

 
Questions.

1.       Formulate as a Mixed Integer Program the problem of ordering the right number of parts and scheduling the right number of trucks.

2.       Discuss what are the key tradeoffs in this problem and how will different parameters influence the nature of the optimal solution.

#### Our Assumption:
1. We order in boxes (in stead of quantity)
2. For different part i, box is in different size, thus could has different quantity per box and different volume per box
3. Different trucks could have different capacity and cost; there are a certain numer of trucks we can schedule from. 
##### (we can also assume box and trucks are identical, which would reduce model complexity but i think that's not reasonable in reality)


## The model
We can formulate the ordering problem as a cost minimization problem. 
Our objection would be to minimize total cost: cost of trucks plus cost of inventory.

Such that:
1. We don't run out of parts, for any parts in any given day
2. We don't run out of truck capacity, for any orderd truck in any day
3. Unused parts would carry forward to the next day



### Variables:

$X_{i,j}: \text{Integer Variable: number of Box we order for part i on day j}\\$

$Y_{j,k}: \text{Binary Variable: if a Truck i is ordered on day j}\\$

$Z_{i,j,k}: \text{Integer Variable: number of Box we order for part i in day j, that is allocated in truck k}\\$

$S_{i,j}: \text{Contineous Variable: quantity of parts as inventory, for parts i at the end of Day j}\\$

### Assume we are given:

$Quantity_{i}: \text{the quantity of parts in a box for part i}\\$

$Volume_{i}: \text{the box volume for part i}\\$

$Capacity_{k}: \text{max capacity for a truck k}\\$

$Cost_{k}: \text{cost of truck k for one task}\\$

$Demand_{j,k}: \text{the quantity demand of part i in day j}\\$



### Goal: Minimize Trucks Cost (plus inventory cost )

$min \sum_{j=1}^{d}\sum_{k=0}^{K}{TruckCost_{k}}{Y_{jk}} + \sum_{i=0}^{N}\sum_{j=0}^{d-1}{InventoryCost_{ij}}{s_{ij}}$


### Constraints:

#### 1. We don't run out of parts, for any parts in any given day
    
${X_{i,j} * {Quantity_{i}} + Inventory_{j-1} > Demand_{i,j} }, \ \forall i, \forall j \\$ 
    
#### 2. We don't run out of truck capacity, for any day, for any ordered Truck
    
$\sum_{i\in{PartsSet}} {Z_{i,j,k} * {BoxVolume_{i}} < TruckVolume_{k} }, \ \forall j, \forall k \\$ 

#### 3. Unused parts would carry forward to the next day, and we want set inventory below a certain amount
    
${Inventory_{i,0} = InitInventory}, \ j=0 \\$

${Inventory_{i,j-1} = X_{i,j-1} * Quantity_{i} + Inventory_{i,j-2} - Demand_{i,j-1} }, \ \forall {j\in {2,3,4,5,...d} }\\$ 
#### 4. Keep Inventory low
${Inventory_{i,j} <= MaxInventory }, \ \forall {j\in {0,1,2,3,4,5,...d} }\\$ 
    
    (or in another way, add inventory as a cost into our objective function)


#### 5. Ordered boxes equal to transported boxes by summing across all ordered trucks
${\sum_{k=0}^{K}{Z_{i.j,k}} == X_{i,j}, \ \forall {i,j} }\\$ 



#### 6. If any box is allocated in a truck, we must order that truck
${Y_{i,j} * Big_M >= Z_{i,j,k} }, \ \forall {i,j,k }\\$ 

            

#### Next We can formulate and solve the problem for a demo

In [30]:
# used packages
# use GLPK as a mip solver
from pulp import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import random

%matplotlib inline

In [53]:
# Init some parameters

## for example say we have 20 different parts
num_part_type = 20
## we can order from 5 trucks
num_truck = 5
## we can add a inventory cost to measure inventory
## notice there is a trade off between inventory cost and truck cost
inventory_cost = 100
## we make a ording plan, for example, Monday to Friday day in next week
plan_range = range(1,6)
## a big m helper for binary indicator
Big_M = 9999999

In [54]:
# init some data for the problem
PartsList = ['Part' + str(i) for i in range(num_part_type)] 
TrucksList = ['Truck' + str(i) for i in range(num_truck)] 
BoxQuantityPerPart= ['BoxQuantityPerPart' + str(i) for i in range(num_part_type)] 
BoxVolumePerPart = ['BoxVolumePerPart' + str(i) for i in range(num_part_type)] 

# quantity of parts in each box, for part i
QuantityDict = {}
for part in PartsList:
    QuantityDict[part] = random.randint(5,10)

# volume of box, for part i  
VolumeDict = {}
for part in PartsList:
    VolumeDict[part] = random.randint(15,20)
    
# capacity of truck, for each different truck    
CapacityDict = {}
for truck in TrucksList:
    CapacityDict[truck] = random.randint(1500,2000)
    
# cost of truck, for each different truck    
CostDict = {}
for truck in TrucksList:
    CostDict[truck] =random.randint(2000,5000)
    
# demand of parts i, on day j, in quantity
DemandDict = {}
for part in PartsList:
    for day in plan_range:
        DemandDict[part,day] = random.randint(60,80)

#initally we set there is no inventory 
#we'll update it as we computing optimal
InventoryDict = {}
for part in PartsList:
    for day in (plan_range):
        InventoryDict[part,day] = 0

### Formulate Problem

In [55]:
#create the problme
prob=LpProblem("Ordering", LpMinimize)

### Declare Variables

In [56]:

#Decision Variable x: integer variable - number of boxes we order for part i in day j
x = pulp.LpVariable.dicts("PartOrder",( (i,j) for i in PartsList for j in plan_range ),lowBound=0, cat=LpInteger)

#Decision Variable y: integer variable - number of trucks we order for day j
y = pulp.LpVariable.dicts("TruckOrder",( (j,k) for j in plan_range for k in TrucksList ), 0, 1, cat=LpBinary)

#Auxilliary Variable y: a integer variable to track number of boxes of part i, ordered on day j, delivered in truck k
z = pulp.LpVariable.dicts("PartInTruck",((i, j, k) for i in PartsList 
                                         for j in plan_range 
                                         for k in TrucksList ),lowBound = 0, cat=LpInteger)

#Auxilliary Variable i: A contineous variable to track Inventory for part i on day j, in the unity of quantity
s = pulp.LpVariable.dicts("Inventory",( (i,j) for i in PartsList for j in [0] + [temp for temp in plan_range] ),lowBound=0, cat=LpContinuous)

### Write Our Objective Function

In [57]:
#obj # Minimize total cost under constriants
prob += lpSum(y[(j,k)]*CostDict[k] for j in plan_range for k in TrucksList) +\
        lpSum(InventoryDict[(i,j)]*inventory_cost for i in PartsList for j in plan_range)

### Write Our Constriants

In [58]:
# [constriant] - Order+Inventory > Demand
## for each part i on each given day j, our new order quantity+Inventory quantity must be greater than demand[i,j]
for i in PartsList:
    for j in plan_range:
        # for the first day, we can set the initial inventory to 0 or any inventory level we have
        if j ==1:
            InventoryDict[i,0]=0 
        else:
            #update inventory level from previous day: inventory = Ordered - Used
            InventoryDict[i,j-1] = x[i,j-1]*QuantityDict[i] + InventoryDict[i,j-2] - DemandDict[i,j-1]
            
        prob += (x[i,j]*QuantityDict[i]) + InventoryDict[i,j-1]  >= DemandDict[i,j]
        prob += s[i,j-1] == InventoryDict[i,j-1]

# [constriant] - keep inventory low
#for i in PartsList:
#    for j in plan_range:
#        prob += InventoryDict[i,j] <= 10
        
# [constriant] - sum of Box Volume in one truck <= Truck Capacity
## for each day, for each truck, the sum of box volume(in that truck) should be less than the capacity
for j in plan_range:
    for k in TrucksList:
        prob += lpSum(z[(i,j,k)]*VolumeDict[i] for i in PartsList) <= CapacityDict[k]

# [constriant] - all ordered parts should be trasported by truck on that day
for i in PartsList:
    for j in plan_range:
        prob += lpSum(z[(i,j,k)] for k in TrucksList) == x[i,j]
        
        
# [constriant] - if any box allocated in a truck, we must order that truck
for i in PartsList:
    for j in plan_range:
        for k in TrucksList:
            prob +=y[j,k]*Big_M >= z[i,j,k]

#### Next we can solve the MIP and get a optimal solution 

In [59]:

#%time prob.solve(GLPK(options=['--mipgap', '0.01']))
%time prob.solve()
print(LpStatus[prob.status])

Wall time: 2min 53s
Optimal


In [60]:
varsdict = {}
for v in prob.variables():
    varsdict[v.name] = v.varValue
#varsdict

#### Next we parse optimal solution to validate model&result

In [61]:
result_inventory_dict = {}
result_order_dict = {}
result_truck_alloc_dict ={}
resulst_truck_order_dict = {}
for k,v in varsdict.items():
    if 'Inventory' in k:
        result_inventory_dict [k.split('_')[1][2:-2], int(k.split('_')[2][0]) ]= v
    if 'PartOrder' in k:
        result_order_dict [k.split('_')[1][2:-2], int(k.split('_')[2][0]) ]= v
    if 'PartInTruck' in k:
        result_truck_alloc_dict [k.split('_')[1][2:-2], int(k.split('_')[2][0]), k.split('_')[3][1:-2] ]= v
    if 'TruckOrder' in k:
        resulst_truck_order_dict [int(k.split('_')[1][1]), k.split('_')[2][1:-2] ]= v


In [62]:
resulst_truck_order_dict

{(1, 'Truck0'): 1.0,
 (1, 'Truck1'): 0.0,
 (1, 'Truck2'): 1.0,
 (1, 'Truck3'): 0.0,
 (1, 'Truck4'): 0.0,
 (2, 'Truck0'): 1.0,
 (2, 'Truck1'): 0.0,
 (2, 'Truck2'): 1.0,
 (2, 'Truck3'): 0.0,
 (2, 'Truck4'): 0.0,
 (3, 'Truck0'): 1.0,
 (3, 'Truck1'): 0.0,
 (3, 'Truck2'): 1.0,
 (3, 'Truck3'): 0.0,
 (3, 'Truck4'): 0.0,
 (4, 'Truck0'): 1.0,
 (4, 'Truck1'): 0.0,
 (4, 'Truck2'): 1.0,
 (4, 'Truck3'): 0.0,
 (4, 'Truck4'): 0.0,
 (5, 'Truck0'): 1.0,
 (5, 'Truck1'): 0.0,
 (5, 'Truck2'): 1.0,
 (5, 'Truck3'): 0.0,
 (5, 'Truck4'): 0.0}

In [63]:
# check currentOrder + inventory > Demand
for part in PartsList:
    for day in plan_range:
        if not (result_order_dict[part, day]*VolumeDict[part] + result_inventory_dict[part,day-1] >= DemandDict[part,day]):
            print('we run out of inventory, something is wrong!')


In [64]:
# check remaining space on trucks

for day in plan_range:
    for truck in TrucksList:
        ordered_vol = 0
        for part in PartsList:
            ordered_vol += result_truck_alloc_dict[part,day,truck]*VolumeDict[part]
        # if a truck is used on one day, check remaning capacity is bigger than 0
        if resulst_truck_order_dict[day,truck] == 1:
            print('unused capacity for day '+ str(day) + ' '+str(truck))
            print(CapacityDict[truck] - ordered_vol)
        # in a truck is not used, remaining capacity should be max capacity
        else:
            if not (CapacityDict[truck] - ordered_box*VolumeDict[part] == CapacityDict[truck]):
                print('truck capacity constraint invalid')

unused capacity for day 1 Truck0
17.0
unused capacity for day 1 Truck2
89.0
unused capacity for day 2 Truck0
11.0
unused capacity for day 2 Truck2
1.0
unused capacity for day 3 Truck0
267.0
unused capacity for day 3 Truck2
116.0
unused capacity for day 4 Truck0
11.0
unused capacity for day 4 Truck2
2.0
unused capacity for day 5 Truck0
17.0
unused capacity for day 5 Truck2
2.0


#### Key Trade offs and Parameter Effects

##### 1. Trade off between model complexity and computing time:
    
    Here we assume all parts and trucks are different, I.e. for different parts, the quantity in a box and volume of the box are different and for a truck, the cost and capacity of truck can vary. Such kinds of assumption could reflect complex real business cases but would require a longer computing time for optimal. Because basically the solver would have to run for all the combination of box-truck allocation.

    However, if we assume all parts are shifted in the same size box with same quantity, and all trucks in the market are identical in capacity and cost, we can quickly reduce model complexity. But such kinds of naive assumption may fail to model reality.
    
    Our model complexity depend on number of parts, number of trucks, and optimal schedule lenth.



##### 2. Trade off between truck's tranporting cost and inventory cost:
    
    We notice there are two optmizing component in the problem: to minimize truck's cost and minimize inventory cost. Tradeoff here would be: if the truck cost is relatively high, our model would tent to use truck's capacity as much as possible, and allow a larger inventory; take a extreme case as an example, if we don't have a inventory cost, the model would useup all capacity and allow inventory for multiple following days.
    
    While if inventory cost is relatively high, our model would tend to solve for a solution that keep inventory low but allow some waste space in ordered truck.

##### 3. Parameter's effect

    Number of parts, number of different trucks, number of different boxes are some important parameters in our problem. While it's generally hard to measure the complexity of a MIP, more variables usually make the problem harder to solve. For example, if we are using a branch and cut algorithm, we would have to do a N choose K hyperplane cut to tight the linear relaxiation, which would quickly increase solving time as N becomes larger. 
    
    Other parameters like Quantity of boxes, Volume of boxes, cost of truck, capacity of truck would also affect optimal solution. For example, if all boxes are relatively large for the truck, it would be hard to squeeze many boxes into one truck, thus our optimal solution will inevitablly have many trucks that have extra unused capacity, and we cannot find any box to fit in. While on the other hand, if there exist some boxes that are relatively small, we would be more efficient in using up truck capacity and thus reduce total truck cost. 
    
    Different kinds of scenarials could happen in reality, and we can take a further step after computing a optimal solution. For example, if we find our optmal solution indeed has a lot of unused capacity in trucks, we could propose to shift parts in some smaller boxes and smaller quantities, which might help to reduce overall cost. (essntially this is we are trying to relax some constriants to get a better objective value)